# 難易度調整
Proof of Work では、極端にマイニングの生成時間が短くなったり、長くなったりしないようにすることを目的に、難易度が定期的に調整される。
## 難易度調整のルール
ビットコインのブロックチェーンの場合、Proof of Workにおける難易度は2016ブロックごとに調整される。  
ブロックは約10分ごとに生成されるようになっているため、2016ブロックはおおよそ２週間。  
難易度調整のための新しい難易度は、理想的な時間である20160分と実際に2016ブロックを生成するのにかかった時間の比率を、古い難易度に掛け算して求める。  
難易度調整は決まったタイミングごとに自動的に実行されるようになっており、調整の幅は最大で４倍、最小で1/4と制限がかけられている。  
マイニングは調整された難易度に則って行わなければならず、ノードがブロックチェーンを検証する際に難易度が正確でないブロックは不正なものとして破棄され、正しい難易度で生成されたブロックのみが採用されるようになっている。
## 難易度調整が行われる理由
現在マイニングに利用されるマシンは著しく性能が高くなっており、それに伴いマイニングに成功する時間感覚が短くなっているが、    
仮に難易度調整が行われなかった場合、マイニング報酬が生成されるスパンが短くなりインフレーションが怒ってしまう可能性が高くなる。  
一方、マイニングに成功する時間感覚が長くなる場合、なかなかトランザクションが承認されないという事態が起こる。  
このようなことを避けるため、安定してほぼ同じ感覚でブロックが生成されるように難易度調整が行われる。
## 難易度調整の仕組みを確認
Blockchainクラスの１つのメソッドとして定義し、確認する。  
今回は５ブロックごとに難易度調整が行われるようにしており、140秒に１回マイニングが成功することを目標としている。  
また、難易度調整を正しく機能させるために、INITIAL _BITSの値を1d777777で行う。  

In [ ]:
def get_retarget_bits(self):
    # 難易度調整をすべきか判断。
    # 本体の長さが０であるか、５の倍数でない場合は　−１　を返り値として返す。
    if len(self.chain) == 0 or len(self.chain) % 5 != 0:
        return -1

    # ブロックチェーンのブロック数が５の倍数になった時、
    # ブロック１つあたりの目標時間に５ブロック分をかけた時間を格納。
    expected_time = 140 * 5

    # 難易度調整を行うためにマイニングにかかった時間を算出
    # ブロックチェーンの長さが５でなかったら、
    if len(self.chain) != 5:
        # 後ろから６番目のfirst_blockとして取り出す
        first_block = self.chain[-(1 + 5)]
    # ブロックチェーンの長さが５だったら
    else:
        # 最初のブロックをfirst_blockとして取り出す
        first_block = self.chain[0]
    # 一番後ろのブロックをlast_blockとして取り出す
    last_block = self.chain[-1]

    # first_blockとlast_blockのそれぞれのタイムスタンプを取得
    first_time = first_block.timestamp.timestamp()
    last_time = last_block.timestamp.timestamp()
    # 実際のマイニングにかかった時間を、last_timeからfirst_timeを引いて経過時間を計算してtotal_timeとする。
    total_time = last_time - first_time

    # last_blockからターゲットを取り出す
    target = last_block.calc_target()
    # マイニングにかかった比を計算
    delta = total_time / expected_time
    # マイニングにかかった比が0.25より小さい時か、4より大きい時はそれぞれ0.25と4でリミッターを設定する。
    if delta < 0.25:
        delta = 0.25
    if delta > 4:
        delta = 4

    # その後、last_blockから取り出したtargetにdeltaを掛けてnew_targetを算出。
    new_target = int(target * delta)

    # # bitsを右に24ビットシフトさせ、3を引いている。
    # Bitsは1e777777といった形で与えられているので右に２４ビットずらすと、exponentに当たる1eだけが残り、そこからcoefficientが3バイトのため後々桁数をずらす計算をする際に生合成を取るために３を引く。
    exponent_bytes = (last_block.bits >> 24) - 3
    # 1バイトが８ビットのため、８を掛けている
    exponent_bits = exponent_bytes * 8

    # exponent_bitsが表すビット数だけnew_targetを右にシフトさせ、一時的なbitsであるtemp_bitsを計算。
    temp_bits = new_target >> exponent_bits

    # exponentは最終的にcoefficientを左に何ビット分シフトさせるかという部分を定義するので、exponentを大きくすれば、算出されるターゲットの値が大きくなりマイニングにおける難易度は下がる。
    if temp_bits != temp_bits & 0xffffff: #大きすぎる
        exponent_bytes += 1
        exponent_bits += 8
    elif temp_bits == temp_bits & 0xffff: #小さすぎる
        exponent_bytes -= 1
        exponent_bits -= 8
    
    # ビット論理話を使ってbitsを計算。この２つのビット論理話を取ることで先頭１バイトがexponent, 後半3バイトがcoefficienceである全４バイトの新しいbitsが計算される。
    # 左辺はexponent_bytesに３を足し、２４ビットだけ左にシフトしており、この時右側に２４ビット(8バイト)分、０が追加される。
    # 右辺はnew_targetをexponent_bitsだけ右にシフトさせ、３バイト分だけ残るcoefficienceを計算。
    return ((exponent_bytes + 3) << 24) | (new_target >> exponent_bits)

In [4]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

# ここのブロックを定義するクラス
class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    # 特殊メソッドの__setitem__を利用し、setattrメソッドで属性を追加
    def __setitem__(self, key, value):
        # pythonでは.(ドット)を使うことで直接オブジェクトにアクセスすることができるが、setattrを利用すると、どのオブジェクトにどのようなkeyとvalueをつけるか指定できる
        setattr(self, key, value)

    def to_json(self):
        return {
            "index": self.index,
            "prev_hash": self.prev_hash,
            "stored_data": self.data,
            "timestamp": self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            # rjust(8, "0")は全部で指定された文字になるように右寄せにするメソッド。この場合、全部で８文字になるように右寄せし、左側の足りない部分は０で埋める処理をする。
            "bits" : hex(self.bits)[2:].rjust(8, "0"),
            "nonce" : hex(self.nonce)[2:],
            "elapsed_time": self.elapsed_time,
            "block_hash": self.block_hash
        }

    # ブロックヘッダを構築したのち、それをSHA256でハッシュ化し、その結果を返す処理
    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + str(self.bits)[2:] + str(self.nonce)# [2:]で先頭２文字を飛ばしているのは、16真数を表す0xが先頭についているため。0xを飛ばしてbitsの値そのものを抜き出すためにソートをしている。
        # hashlibは文字列をエンコードしたものを引数として渡す必要があるため、上の行のようみ全てを文字列に変換したものを.encode()によってエンコードしている。
        h = hashlib.sha256(blockheader.encode()).hexdigest()# .encode()は引数を設定しなければでジョルとでUTF-8が使われる。
        # 冒頭で定義したブロックハッシュの変数に上の行で計算したハッシュ値を代入
        self.block_hash = h
        return h

    # 与えられたbitsからtargetを算出
    def calc_target(self):
        # bitsを右に24ビットシフトさせ、3を引いている。
        # Bitsは1e777777といった形で与えられているので右に２４ビットずらすと、exponentに当たる1eだけが残り、そこからcoefficientが3バイトのため後々桁数をずらす計算をする際に生合成を取るために３を引く。
        exponent_bytes = (self.bits >> 24) - 3
        # 1バイトが８ビットのため、８を掛けている
        exponent_bits = exponent_bytes * 8
        # bitsからcoefficientを抽出するために行う。
        # &はビット論理積で両方が１の時のみ1になり、それ以外は0となるため、ビット論理積をとれば相手側が１の部分のみが残って、桁数が同じであれば同じ結果になる。
        # 加えて、桁数が6桁で、bitsが8桁のため先頭2桁の1eを排除することができ、結果としてcoefficientの部分のみが抽出できる。
        coefficient = self.bits & 0xffffff
        # coefficientをexponent_bits分、左にシフトさせることでターゲットが算出され、返り値として返される。
        return coefficient << exponent_bits

    # 計算されたハッシュ値が先ほど計算したターゲットよりも小さいかどうかを判定している。
    def check_valid_hash(self):
        # 左側の「~~16)」はハッシュ値を１６真数であると見立てて１０進数の整数に変換するための記述で、これによってハッシュ値とターゲットを比較できるようになる。
        return int(self.calc_blockhash(), 16) <= self.calc_target()


# ブロックの関係性を定義するクラス
class BlockChain():
    def __init__(self, initial_bits):
        # 変数としてブロックの情報を格納するからの配列
        self.chain = []
        # 難易度を定義するbits
        self.initial_bits = initial_bits

    # chainの配列にブロックのデータを追加するメソッド
    def add_block(self, block):
        self.chain.append(block)

    # chainの配列のその時点での最後の要素を取り出し、json形式で出力する
    def getblockinfo(self, index=-1):
        # json.dumpsは辞書型をJSON形式に変換するメソッド。PythonではJSON形式は文字列型として扱われる。
        # 第１引数は変換したい辞書型のデータを指定し、
        # 第２引数ではインデントの大きさ、
        # 第３引数では辞書の出力がキーでソートできるようにTrueを指定、
        # 第４引数ensure_ascii=Falseは日本語表記出力結果がひと続きで出力されるので、可読性を高めるためにいJSON形式で出力する。
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))

    # ブロックをつなげていくための処理
    def mining(self, block):
        # 処理がスタートした時間を取り出している。
        start_time = int(time.time() * 1000)
        # 条件が一致している間はそこから続くfor文を伊h他すら続けていく処理
        while True:
            # MAX_32BIT + 1 と１を足しているのは、range関数が０からカウントを始めるため。１を足すことで最大値であるMAX_32BITまで代入できる。
            for n in range(MAX_32BIT + 1):
                # Nonceを次々と更新
                block.nonce = n
                # Blockクラスで実装しているcheck_valid_hashの結果がTRUE、つまりターゲットより小さいハッシュ値が見つかれば、
                if block.check_valid_hash():
                    # 終了した時間と経過時間などを計算し、
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = \
                    str((end_time - start_time) / \
                        1000.0) + "秒"
                    # chainに追加
                    self.add_block(block)
                    # ブロックの情報を出力
                    self.getblockinfo()
                    return
            # ブロックのタイムスタンプと終了時のタイムスタンプが同じだった場合の処理を定義している。
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    # ジェネシスブロックを生成するためのメソッド
    # ジェネシスブロックはブロックチェーンにおける最初のブロックで他のブロックとは「prebhashの値を指定しないと参照する値がないという点」で異なっている。
    def create_genesis(self):
        # Blockクラスの引数にジェネシスブロックの情報を指定することで、genesis_blockインスタンスを生成。
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        # マイニングを行い、ジェネシスブロックの生成を行う。
        self.mining(genesis_block)

    def get_retarget_bits(self):
        # 難易度調整をすべきか判断。
        # 本体の長さが０であるか、５の倍数でない場合は　−１　を返り値として返す。
        if len(self.chain) == 0 or len(self.chain) % 5 != 0:
            return -1

        # ブロックチェーンのブロック数が５の倍数になった時、
        # ブロック１つあたりの目標時間に５ブロック分をかけた時間を格納。
        expected_time = 140 * 5

        # 難易度調整を行うためにマイニングにかかった時間を算出
        # ブロックチェーンの長さが５でなかったら、
        if len(self.chain) != 5:
            # 後ろから６番目のfirst_blockとして取り出す
            first_block = self.chain[-(1 + 5)]
        # ブロックチェーンの長さが５だったら
        else:
            # 最初のブロックをfirst_blockとして取り出す
            first_block = self.chain[0]
        # 一番後ろのブロックをlast_blockとして取り出す
        last_block = self.chain[-1]

        # first_blockとlast_blockのそれぞれのタイムスタンプを取得
        first_time = first_block.timestamp.timestamp()
        last_time = last_block.timestamp.timestamp()
        # 実際のマイニングにかかった時間を、last_timeからfirst_timeを引いて経過時間を計算してtotal_timeとする。
        total_time = last_time - first_time

        # last_blockからターゲットを取り出す
        target = last_block.calc_target()
        # マイニングにかかった比を計算
        delta = total_time / expected_time
        # マイニングにかかった比が0.25より小さい時か、4より大きい時はそれぞれ0.25と4でリミッターを設定する。
        if delta < 0.25:
            delta = 0.25
        if delta > 4:
            delta = 4

        # その後、last_blockから取り出したtargetにdeltaを掛けてnew_targetを算出。
        new_target = int(target * delta)

        # # bitsを右に24ビットシフトさせ、3を引いている。
        # Bitsは1e777777といった形で与えられているので右に２４ビットずらすと、exponentに当たる1eだけが残り、そこからcoefficientが3バイトのため後々桁数をずらす計算をする際に生合成を取るために３を引く。
        exponent_bytes = (last_block.bits >> 24) - 3
        # 1バイトが８ビットのため、８を掛けている
        exponent_bits = exponent_bytes * 8

        # exponent_bitsが表すビット数だけnew_targetを右にシフトさせ、一時的なbitsであるtemp_bitsを計算。
        temp_bits = new_target >> exponent_bits

        # exponentは最終的にcoefficientを左に何ビット分シフトさせるかという部分を定義するので、exponentを大きくすれば、算出されるターゲットの値が大きくなりマイニングにおける難易度は下がる。
        if temp_bits != temp_bits & 0xffffff: #大きすぎる
            exponent_bytes += 1
            exponent_bits += 8
        elif temp_bits == temp_bits & 0xffff: #小さすぎる
            exponent_bytes -= 1
            exponent_bits -= 8
        
        # ビット論理話を使ってbitsを計算。この２つのビット論理話を取ることで先頭１バイトがexponent, 後半3バイトがcoefficienceである全４バイトの新しいbitsが計算される。
        # 左辺はexponent_bytesに３を足し、２４ビットだけ左にシフトしており、この時右側に２４ビット(8バイト)分、０が追加される。
        # 右辺はnew_targetをexponent_bitsだけ右にシフトさせ、３バイト分だけ残るcoefficienceを計算。
        return ((exponent_bytes + 3) << 24) | (new_target >> exponent_bits)

    # 最新のブロックの情報を引数として利用することで、次々と鎖上につなげていくことが可能
    def add_newblock(self, i):
        # ブロックチェーンの本体情報であるchainの配列の一番後ろのブロックをlast_blockとして取り出し、それを引数にBlockクラスをインスタンス化する
        last_block = self.chain[-1]

        # まず、new_bitsとしてget_retarget_bitsメソッドの結果を格納。この時のnew_bitsは新しいbitsか−1のいずれかが格納されている。
        new_bits = self.get_retarget_bits()

        # new_bitsが0より小さい、つまり-1だった場合,
        if new_bits < 0:
            # 一つ前のブロックの難易度を適用
            bits = last_block.bits
        # new_bitsが0より大きい、つまり、新しいbitsだった場合、
        else:
            # new_bitsを適用
            bits = new_bits

        # last_blockのblock_hashとbitsをそれぞれ引数として指定。
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), bits)
        self.mining(block)

In [5]:
if __name__ == "__main__":
    # ブロックチェーンクラスをインスタンス化
    bc = BlockChain(INITIAL_BITS)
    print("ジェネシスブロックを作成中 ・・・")
    # ジェネシスブロックを生成
    bc.create_genesis()
    # 30個の新規ブロックの生成
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中 ・・・
{
  "bits": "1e777777",
  "block_hash": "000020dff7f8ee6e8f22ac6a1eab7d9b3ddd3d5a50f8ca71bf57e976dacca114",
  "elapsed_time": "0.51秒",
  "index": 0,
  "nonce": "1419d",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2022/06/14 14:34:02"
}
2番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00007502bced15a5a973b411e9c8ad8ec748ea4333ee8b0affb8bcb33063d969",
  "elapsed_time": "0.858秒",
  "index": 1,
  "nonce": "23b7b",
  "prev_hash": "000020dff7f8ee6e8f22ac6a1eab7d9b3ddd3d5a50f8ca71bf57e976dacca114",
  "stored_data": "ブロック 1",
  "timestamp": "2022/06/14 14:34:02"
}
3番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000060bfeeaa633040d480661d69323db566ba50cb7c10f097ab0147f02d3c52",
  "elapsed_time": "1.42秒",
  "index": 2,
  "nonce": "3cd3e",
  "prev_hash": "00007502bced15a5a973b411e9c8ad8ec748ea4333ee8b0affb8bcb33063d969",
  "stored_data": "ブロック 2",
  "timestamp": "2022/06/14 14:3